# Normalization

## Rules implemented:
* Іі Ѣѣ Ѳѳ Ѵѵ --> Ии Ее Фф Ии
* jeja

## todo
* Final tverdyj znak
* prefixes in z
* ago, jago
* yja, ija
* o after palatals
* e oborotnoe before i

In [7]:
import re

def normalizeRus(inputtext):
    # Replace Іі Ѣѣ Ѳѳ Ѵѵ
    oldstring = "ІіѢѣѲѳѴѵ"
    newstring = "ИиЕеФфИи"
    translationtable = str.maketrans(oldstring, newstring)
    
    # jeja|ея|ЕЯ
    eja = re.sub(r'\bЕЯ\b', 'ЕЕ', re.sub(r'\bея\b', 'ее', re.sub(r'\bЕя\b', 'Ее' ,inputtext)))
    
    
    
    return eja.translate(translationtable)

    
    

print(normalizeRus("БѢЛАЯ АКАЦІЯ Ея ваываЕяавав ея вававеявпап ЕЯ ВЕЯВ ЕЯВ"))

БЕЛАЯ АКАЦИЯ Ее ваываЕяавав ее вававеявпап ЕЕ ВЕЯВ ЕЯВ
